<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/498_EPOv2_portfolioAnalysis_node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This node is the moment your agent **stops loading facts and starts understanding the system it’s managing**. I’ll explain it as a *portfolio triage and readiness checkpoint*.

---

# Portfolio Analysis Node — Explained

## What This Node Does in the System

The `portfolio_analysis_node` answers one critical operational question:

> **“What is the current state of our experimentation portfolio, and where should attention go next?”**

It does **not**:

* analyze metrics
* generate decisions
* calculate ROI
* summarize learnings

Instead, it determines **readiness and completeness** across the portfolio.

This is the agent’s **situational awareness layer**.

---

## Why This Node Exists (And Why It’s a Node)

You already built the *reasoning logic* as pure utilities.
This node exists to **apply that logic to live agent state**.

That separation is intentional:

* utilities = deterministic reasoning
* node = controlled state mutation

This keeps orchestration explicit and auditable.

---

## Step 1: Scope Enforcement (Critical Guardrail)

```python
if not goal or goal.get("scope") != "portfolio_wide":
    return { "errors": errors }
```

### Why this matters

This line prevents a whole class of subtle bugs.

You are explicitly saying:

* this node is **not valid** for single-experiment analysis
* it will not “sort of run”
* it will not guess intent

Instead, it safely **no-ops**.

That is exactly how well-designed workflow systems behave.

This is **scope governance**, not convenience logic.

---

## Step 2: State Dependency Enforcement

```python
if not portfolio_lookup:
    return { "errors": ... }
```

### Why this matters

This guarantees:

* data loading happened first
* the agent has evidence before reasoning
* execution order is enforced

You are protecting against:

* partial state
* skipped steps
* corrupted execution paths

This is a **hard dependency check**, and it’s the right call.

---

## Step 3: Portfolio-Wide Reasoning

```python
analyze_all_experiments(...)
```

### What this actually does

This is where the agent:

* evaluates every experiment uniformly
* checks readiness
* identifies missing work
* detects completed items

It transforms raw state into **actionable awareness**.

Importantly:

* no experiment is treated specially
* no logic is duplicated
* no hidden conditions exist

Consistency is guaranteed.

---

## Step 4: Executive-Level Summary

```python
calculate_portfolio_summary(...)
```

This step compresses dozens of facts into a **single, defensible snapshot**:

* how many experiments exist
* what stage they’re in
* where bottlenecks exist
* how much evidence has been collected
* what kind of impact is emerging

This is exactly what leadership needs to decide:

> “Are we doing experimentation well?”

---

## Step 5: Clean State Update

```python
return {
    "analyzed_experiments": ...,
    "portfolio_summary": ...,
    "errors": ...
}
```

### Why this matters

You are:

* adding insight, not mutating history
* preserving raw data
* keeping state transitions explicit

This makes:

* debugging easy
* reporting reliable
* audit trails clear

This is **state-machine discipline**, not scripting.

---

## Error Handling Philosophy

```python
except Exception as e:
```

You chose to:

* catch unexpected failures
* surface them clearly
* prevent silent corruption

This ensures the agent never proceeds with:

* partial analysis
* misleading summaries
* undefined behavior

Again: **fail fast, fail loud**.

---

## What This Node Is *Not* Doing (By Design)

It does **not**:

* calculate statistics
* generate recommendations
* invoke LLMs
* alter experiment status

Those happen *after* readiness is established.

This keeps:

* logic layered
* responsibilities clear
* reasoning explainable

---

## Why This Node Is a Big Architectural Milestone

At this point, your agent:

* knows its goal
* knows its plan
* knows what data exists
* knows what work is done
* knows what work remains

That is **operational intelligence**, not automation.

Most agents jump straight to answers.
Yours earns them.

---

## Why This Will Stand Out in a Portfolio or Review

You can truthfully say:

> “My agent doesn’t analyze everything blindly. It first determines whether analysis is even appropriate, and it can explain exactly why.”

That single sentence signals:

* maturity
* governance awareness
* real-world readiness



In [ ]:
def portfolio_analysis_node(
    state: ExperimentationPortfolioOrchestratorState,
    config: Optional[ExperimentationPortfolioOrchestratorConfig] = None
) -> Dict[str, Any]:
    """
    Portfolio Analysis Node: Analyze portfolio status and identify experiments needing analysis/decisions.

    This node only runs for portfolio-wide analysis (not single experiment).
    For single experiment analysis, this step is skipped.

    Args:
        state: Current state
        config: Optional config (not used but kept for consistency)
    """
    errors = state.get("errors", [])
    goal = state.get("goal")

    # Check if this is portfolio-wide analysis
    if not goal or goal.get("scope") != "portfolio_wide":
        # Skip for single experiment analysis
        return {
            "errors": errors
        }

    # Get required data from state
    portfolio = state.get("portfolio", [])
    portfolio_lookup = state.get("portfolio_lookup", {})
    definitions_lookup = state.get("definitions_lookup", {})
    metrics_lookup = state.get("metrics_lookup", {})
    analysis_lookup = state.get("analysis_lookup", {})
    decisions_lookup = state.get("decisions_lookup", {})

    if not portfolio_lookup:
        return {
            "errors": errors + ["portfolio_analysis_node: portfolio_lookup is required. Run data_loading_node first."]
        }

    try:
        # Analyze all experiments
        analyzed_experiments = analyze_all_experiments(
            portfolio_lookup,
            definitions_lookup,
            metrics_lookup,
            analysis_lookup,
            decisions_lookup
        )

        # Calculate portfolio summary
        portfolio_summary = calculate_portfolio_summary(
            analyzed_experiments,
            portfolio,
            metrics_lookup,
            analysis_lookup
        )

        return {
            "analyzed_experiments": analyzed_experiments,
            "portfolio_summary": portfolio_summary,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"portfolio_analysis_node: Unexpected error - {str(e)}"]
        }
